# MNIST CNN Autoencoder Example
Uses Conv2D and Deconv2D operations to create a simple auto-encoder. In this example, it is possible that it learns the trivial function. It's intention is more to see how the *tt.network.conv2d_transpose()* function works.

In [ ]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

In [ ]:
FRAME_W_H = 28
BATCH_SIZE = 64
MAX_STEPS = 2000
LEARNING_RATE = 1e-4

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
sess = tf.InteractiveSession()

### Inference

In [ ]:
x  = tf.placeholder(tf.float32, [BATCH_SIZE, FRAME_W_H ** 2], "X")
y_ = tf.placeholder(tf.float32, [BATCH_SIZE, FRAME_W_H ** 2], "Y_")

x_image = tf.reshape(x, [-1, FRAME_W_H, FRAME_W_H, 1])

with tf.variable_scope("Encoder"):
    # 1: Conv
    conv1 = tt.network.conv2d("Conv1", x_image,
                              32, 5, 5, 1, 1,
                              stddev=0.02, bias=0.1,
                              activation=tf.nn.relu)
    h_pool1 = tt.network.max_pool2d(conv1, 2, 2, 2, 2)

    # 2: Conv
    conv2 = tt.network.conv2d("Conv2", h_pool1,
                              64, 5, 5, 1, 1,
                              stddev=0.02, bias=0.1,
                              activation=tf.nn.relu)
    h_pool2 = tt.network.max_pool2d(conv2, 2, 2, 2, 2)
    encoder_out = h_pool2

with tf.variable_scope("Decoder"):
    # 3: Deconv
    conv3t = tt.network.conv2d_transpose("Deconv1", encoder_out,
                                         32, BATCH_SIZE,
                                         5, 5, 2, 2,
                                         stddev=0.02, bias=0.1,
                                         activation=tf.nn.relu)

    # 4: Deconv
    conv4t = tt.network.conv2d_transpose("Deconv2", conv3t,
                                         1, BATCH_SIZE,
                                         5, 5, 2, 2,
                                         stddev=0.02, bias=0.1)
    decoder_out = conv4t

output = tf.reshape(decoder_out, [-1,FRAME_W_H ** 2])

In [ ]:
with tf.name_scope("Train"):
    loss = tf.reduce_mean(tf.square(y_ - output))
    train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

sess.run(tf.initialize_all_variables())

In [ ]:
tt.visualization.show_graph(sess.graph_def)

### Training

In [ ]:
for i in range(MAX_STEPS + 1):
    batch_images, _ = mnist.train.next_batch(BATCH_SIZE)
    if i % 100 == 0:
        train_loss = loss.eval(feed_dict={
                               x: batch_images,
                               y_: batch_images})
        print("step %d / %d, loss %g" % (i, MAX_STEPS, train_loss))
    
    train_step.run(feed_dict={x: batch_images, y_: batch_images})

### Evaluation

In [ ]:
test_images = mnist.test.images[:BATCH_SIZE]

# print loss
print("test loss %g" % loss.eval(feed_dict={
                                 x: test_images,
                                 y_: test_images}))

In [ ]:
reshaped_test_images = np.reshape(test_images, [-1, FRAME_W_H, FRAME_W_H, 1])
out_images = output.eval(feed_dict={x: test_images})
reshaped_out_images = np.reshape(out_images, [-1, FRAME_W_H, FRAME_W_H, 1])

img = reshaped_test_images[1] * 255
tt.visualization.display_array(img)

tt.visualization.display_batch(reshaped_test_images, 4, 4)

tt.visualization.display_batch(reshaped_out_images, 4, 4)

In [ ]:
img1 = np.ones([240, 320, 1]) * 10
img2 = np.ones([240, 320, 1]) * 110
img3 = np.ones([240, 320, 1]) * 210

img_batch = np.ones([4, 240, 320, 1]) * 210

images = [img1, img2, img3]

img_batch = np.asarray([img1, img2, img3, img1, img2])

tt.visualization.display_array(img1)

tt.visualization.display_batch(images)

tt.visualization.display_batch(img_batch)